In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#%matplotlib inline

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [ ]:
import os

from collections import Counter

from ochre.utils import align_characters

def get_sequences(fname):
    sentences = []
    with open(fname) as f:
        for sentence in f:
            #print(sentence)
            characters = sentence.split()
            #print(characters)
            s = []
            for c in characters:
                if c == '<SPACE>':
                    s.append(' ')
                else:
                    s.append(c)
            #print(''.join(s))
            sentences.append(s)
    return sentences


def convert_to_text(gs_file, ocr_file, pred_file, empty_char='\n'):
    # get sequences for all three input files
    gs_seqs = get_sequences(gs_file)
    ocr_seqs = get_sequences(ocr_file)
    pred_seqs = get_sequences(pred_file)
    
    idx = 0
    counters = {}

    # skip krijg je van loopen over de ocr-input en gs-output voor een file:
    # als ze allebei een spatie hebben, neem de index + 1 (behalve voor 0)
    for j, (s1, s2, s3) in enumerate(zip(gs_seqs, ocr_seqs, pred_seqs)):
        # the final sequence are empty, edlib can't deal with this
        if s3 != [] and s2 != []:
            #print(''.join(s1))
            #print(''.join(s2))
            #print(''.join(s3))
            #print('---')
        
            # align pred with ocr (input)
            ocr, pred, _match = align_characters(s3, s2, empty_char=empty_char)
        
            while len(pred) < len(ocr):
                pred.append(empty_char)
    
            #print(''.join(ocr))
            #print(''.join(pred))
    
            for i, (inp, outp) in enumerate(zip(ocr, pred)):
                if not idx + i in counters.keys():
                    counters[idx+i] = Counter()
                counters[idx+i][outp] += 1
                #print(idx+i, counters[idx+i])    
            
            for i, (c1, c2) in enumerate(zip(s1, s2)):
                # We need the index of the first space in ocr and gs
                if c1 == ' ' and c2 == ' ':
                    skip = i + 1
                    #print('skip', skip)
                    break
            try:
                idx += skip
            except:
                print('  GS', ''.join(s1))
                print(' OCR', ''.join(s2))
                print('PRED', ''.join(s3))
                print('---')
                print(len(s2))
                idx += len(s2)
                
    
            #print()
    
    agg_out = []
    for idx, c in counters.items():
        agg_out.append(c.most_common(1)[0][0])

    corrected_text = u''.join(agg_out)
    corrected_text = corrected_text.replace(empty_char, u'')
    
    return corrected_text      

gs_file = '/home/jvdzwaan/data/kb-ocr/A8P1/test/DDD_010265237_001.gs'
ocr_file = '/home/jvdzwaan/data/kb-ocr/A8P1/test/DDD_010265237_001.ocr'
pred_file = '/home/jvdzwaan/data/kb-ocr/A8P1/pred/DDD_010265237_001.pred'

convert_to_text(gs_file, ocr_file, pred_file)


In [ ]:
import glob

from nlppln.utils import create_dirs, out_file_name

def convert_dir_to_text(test_dir, pred_dir, out_dir):
    create_dirs(out_dir)
    
    gs_files = sorted(glob.glob(f'{test_dir}/*.gs'))
    ocr_files = sorted(glob.glob(f'{test_dir}/*.ocr'))
    pred_files = sorted(glob.glob(f'{pred_dir}/*.pred'))
    
    
    for gs, ocr, pred in zip(gs_files, ocr_files, pred_files):
        print(gs)
        #print(ocr)
        #print(pred)
        
        corrected_text = convert_to_text(gs, ocr, pred)
        
        # The ocrevaluation tool wants *.txt
        out_file = out_file_name(out_dir, pred, ext='txt')
        #print(out_file)
        with open(out_file, 'w') as f:
            f.write(corrected_text)

test_dir = '/home/jvdzwaan/data/kb-ocr/A8P1/test/'
pred_dir = '/home/jvdzwaan/data/kb-ocr/A8P1/pred/'

out_dir = '/home/jvdzwaan/data/kb-ocr/text_aligned_blocks-match_gs/pred-A8P1/'

convert_dir_to_text(test_dir, pred_dir, out_dir)